# 【Kaggle】住宅価格予測

# データセット

In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [3]:
print(train.shape,test.shape)

(1460, 81) (1459, 80)


In [4]:
train=train.drop('Alley',axis=1).drop('FireplaceQu',axis=1).drop('PoolQC',axis=1).drop('Fence',axis=1).drop('MiscFeature',axis=1)

test=test.drop('Alley',axis=1).drop('FireplaceQu',axis=1).drop('PoolQC',axis=1).drop('Fence',axis=1).drop('MiscFeature',axis=1)

In [5]:
train_id = train['Id']
test_id = test['Id']#テストセットのIDは提出用のファイルを作る際に必要

y_train = train['SalePrice']
x_train = train.drop(['Id','SalePrice'], axis=1)
x_test = test.drop('Id', axis=1)

In [6]:
x_train = x_train.fillna(x_train.median())
x_test = x_test.fillna(x_test.median())

x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [7]:
for i in range(x_train.shape[1]):
    if x_train.iloc[:,i].dtypes == object:
        mode = x_train.mode()[x_train.columns.values[i]].values
        for j in range(x_train.shape[0]):
            if x_train.isnull().iloc[j,i]==True:
                x_train.iloc[j,i] = mode

In [8]:
for i in range(x_test.shape[1]):
    if x_test.iloc[:,i].dtypes == object:
        mode = x_test.mode()[x_test.columns.values[i]].values
        for j in range(x_test.shape[0]):
            if x_test.isnull().iloc[j,i]==True:
                x_test.iloc[j,i] = mode

In [9]:
x_train.isnull().sum().sum()

0

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#ラベルエンコーダー(訓練セット）
for i in range(x_train.shape[1]):
    if x_train.iloc[:,i].dtypes == object:
        le.fit(list(x_train[x_train.columns.values[i]].values)) 
        x_train[x_train.columns.values[i]] = le.transform(list(x_train[x_train.columns.values[i]].values))

#ラベルエンコーダー(テストセット）
for i in range(x_test.shape[1]):
    if x_test.iloc[:,i].dtypes == object:
        le.fit(list(x_test[x_test.columns.values[i]].values)) 
        x_test[x_test.columns.values[i]] = le.transform(list(x_test[x_test.columns.values[i]].values))

x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   int64  
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   int64  
 5   LotShape       1460 non-null   int64  
 6   LandContour    1460 non-null   int64  
 7   Utilities      1460 non-null   int64  
 8   LotConfig      1460 non-null   int64  
 9   LandSlope      1460 non-null   int64  
 10  Neighborhood   1460 non-null   int64  
 11  Condition1     1460 non-null   int64  
 12  Condition2     1460 non-null   int64  
 13  BldgType       1460 non-null   int64  
 14  HouseStyle     1460 non-null   int64  
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [24]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=5)
selector.fit(x_train,y_train)

print(selector.get_support())

[False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False  True  True
 False False False False False False False False False False False False
 False False]


In [21]:
print(x_test.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF'

In [28]:
x_train_selected=pd.DataFrame({'OverallQual':x_train['OverallQual'],'ExterQual':x_train['ExterQual'],'GrLivArea':x_train['GrLivArea'],'GarageCars':x_train['GarageCars'],'GarageArea':x_train['GarageArea']})
x_test_selected=pd.DataFrame({'OverallQual':x_test['OverallQual'],'ExterQual':x_test['ExterQual'],'GrLivArea':x_test['GrLivArea'],'GarageCars':x_test['GarageCars'],'GarageArea':x_test['GarageArea']})

x_train_selected.head()

,OverallQual,ExterQual,GrLivArea,GarageCars,GarageArea
0,7,2,1710,2,548
1,6,3,1262,2,460
2,7,2,1786,2,608
3,7,3,1717,3,642
4,8,2,2198,3,836


# モデルの訓練

In [30]:
from sklearn.model_selection import train_test_split
xp_train,xp_test,yp_train,yp_test=train_test_split(x_train_selected,y_train,test_size=0.3,random_state=1) #セットのうち3割をテストセットにする

In [31]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

forest=RandomForestRegressor()
svr=SVR()

parameters_forest={'n_estimators':[100,500,1000,3000],'max_depth':[3,6,12]}
parameters_svr={'C':[0.1,10,1000],'epsilon':[0.01,0.1,0.5]}

In [32]:
from sklearn.model_selection import GridSearchCV
# ランダムフォレスト
grid_forest = GridSearchCV(forest,parameters_forest)
grid_forest.fit(xp_train,yp_train)

# SVR
grid_svr = GridSearchCV(svr,parameters_svr)
grid_svr.fit(xp_train,yp_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 10, 1000], 'epsilon': [0.01, 0.1, 0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [33]:
from sklearn.metrics import mean_squared_error
yp_pred_forest=grid_forest.predict(xp_test)
print(mean_squared_error(yp_test,yp_pred_forest))

990700426.5112605


In [34]:
yp_pred_svr=grid_svr.predict(xp_test)
print(mean_squared_error(yp_test,yp_pred_svr))

4096200933.482609


In [35]:
grid_forest.best_params_

{'max_depth': 6, 'n_estimators': 100}

In [36]:
best_forest=RandomForestRegressor(max_depth= 6, n_estimators=100)
best_forest.fit(x_train_selected,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [37]:
result = np.array(best_forest.predict(x_test_selected))
# 提出用ファイルの作成
df_result=pd.DataFrame(result,columns=['SalePrice'])
df_result=pd.concat([test_id,df_result],axis=1)
df_result.to_csv('houseprices.csv', index=False)